In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration and Setup ---
# UPDATED PATH: Using the path and filename provided by the user
DATA_PATH = 'data/raw/MachineLearningRating_v3.txt' 
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.sans-serif'] = ['Inter'] # Using Inter font for aesthetics

def load_data(path):
    """Loads data, handles basic parsing (using '|' delimiter), and cleans column names."""
    try:
        # UPDATED: Using pipe '|' as the separator
        df = pd.read_csv(path, sep='|')
        
        # Clean column names (e.g., remove spaces and special characters)
        df.columns = df.columns.str.replace(r'[^A-Za-z0-9_]+', '', regex=True)
        df.columns = df.columns.str.replace(r'(.)([A-Z][a-z]+)', r'\1_\2', regex=True).str.lower()
        
        # Strip leading/trailing whitespace from all string columns after loading
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].astype(str).str.strip()
            
        print("Data loaded, pipe-delimiter handled, and columns cleaned successfully.")
        return df
    except FileNotFoundError:
        print(f"ERROR: Data file not found at {path}. Please ensure the file exists in the correct directory.")
        return None
    except Exception as e:
        print(f"An error occurred during data loading: {e}")
        return None

def perform_eda(df):
    """Performs required EDA and calculates key metrics."""
    print("\n--- 1. Data Structure and Quality Assessment ---")
    print(f"Dataset Shape: {df.shape}")
    print("\nMissing Values:")
    print(df.isnull().sum()[df.isnull().sum() > 0].sort_values(ascending=False))
    print("\nData Types:")
    print(df.dtypes.value_counts())

    # Ensure financial columns are numeric
    # These columns must exist after the column cleaning (e.g., TotalPremium -> total_premium)
    df['total_premium'] = pd.to_numeric(df['total_premium'], errors='coerce')
    df['total_claims'] = pd.to_numeric(df['total_claims'], errors='coerce')
    df['custom_value_estimate'] = pd.to_numeric(df['custom_value_estimate'], errors='coerce')
    
    # Drop rows where critical financial data could not be parsed or is missing
    df = df.dropna(subset=['total_premium', 'total_claims'])

    # Feature Engineering for EDA
    df['has_claim'] = (df['total_claims'] > 0).astype(int)
    # Note: Using np.where to handle division by zero for policies with 0 premium.
    df['loss_ratio'] = np.where(df['total_premium'] > 0, df['total_claims'] / df['total_premium'], np.nan) 
    df['transaction_date'] = pd.to_datetime(df['transaction_month']) # Using TransactionMonth as the transaction date
    df['transaction_month'] = df['transaction_date'].dt.to_period('M')

    print("\n--- 2. Descriptive Statistics ---")
    print(df[['total_premium', 'total_claims', 'custom_value_estimate']].describe())

    print("\n--- 3. Key Insight Calculations (Loss Ratio) ---")
    overall_lr = df['total_claims'].sum() / df['total_premium'].sum()
    print(f"Overall Loss Ratio (TotalClaims / TotalPremium): {overall_lr:.4f}")

    # Guiding Question 1: Loss Ratio by key segment
    segments = ['province', 'vehicle_type', 'gender']
    for seg in segments:
        agg_data = df.groupby(seg).agg(
            total_premium=('total_premium', 'sum'),
            total_claims=('total_claims', 'sum')
        ).reset_index()
        # Calculate loss ratio, handling zero premium case
        agg_data['loss_ratio'] = np.where(agg_data['total_premium'] > 0, agg_data['total_claims'] / agg_data['total_premium'], np.nan)
        agg_data = agg_data.dropna(subset=['loss_ratio'])
        print(f"\nLoss Ratio by {seg.capitalize()}:")
        print(agg_data.sort_values(by='loss_ratio', ascending=False))

    # Guiding Question 4: Temporal Trends
    monthly_trend = df.groupby('transaction_month').agg(
        total_premium=('total_premium', 'sum'),
        total_claims=('total_claims', 'sum'),
        num_policies=('policy_id', 'count'),
        num_claims=('has_claim', 'sum')
    ).reset_index()
    monthly_trend['claim_frequency'] = monthly_trend['num_claims'] / monthly_trend['num_policies']
    monthly_trend['loss_ratio'] = np.where(monthly_trend['total_premium'] > 0, monthly_trend['total_claims'] / monthly_trend['total_premium'], np.nan)
    print("\nMonthly Trend Analysis (Claim Frequency and Loss Ratio):")
    print(monthly_trend)

    # Visualization
    print("\n--- 4. Visualizations ---")
    
    # Plot 1: Monthly Loss Ratio and Claim Frequency Trend
    fig, ax1 = plt.subplots()
    ax1.plot(monthly_trend['transaction_month'].astype(str), monthly_trend['loss_ratio'], color='blue', label='Loss Ratio')
    ax1.set_xlabel('Transaction Month')
    ax1.set_ylabel('Loss Ratio', color='blue')
    ax1.tick_params(axis='y', labelcolor='blue')
    ax2 = ax1.twinx() # Secondary y-axis
    ax2.plot(monthly_trend['transaction_month'].astype(str), monthly_trend['claim_frequency'], color='red', linestyle='--', label='Claim Frequency')
    ax2.set_ylabel('Claim Frequency', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    plt.title('Monthly Loss Ratio and Claim Frequency Over Time')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('eda_plot_1_temporal_trend.png')
    plt.close()

    # Plot 2: Distribution of Claims (Outlier Detection)
    # Filter to only policies with claims to see severity distribution
    claim_severity_df = df[df['total_claims'] > 0].copy()
    plt.figure()
    sns.boxplot(x=claim_severity_df['total_claims'])
    plt.title('Box Plot of Claim Severity (TotalClaims > 0)')
    plt.xlabel('Total Claims (ZAR)')
    plt.xscale('log') # Use log scale to handle extreme outliers for better visualization
    plt.savefig('eda_plot_2_claim_severity_distribution.png')
    plt.close()

    # Plot 3: Loss Ratio by Province
    province_lr = df.groupby('province').agg(
        total_premium=('total_premium', 'sum'),
        total_claims=('total_claims', 'sum')
    ).reset_index()
    province_lr['loss_ratio'] = np.where(province_lr['total_premium'] > 0, province_lr['total_claims'] / province_lr['total_premium'], np.nan)
    province_lr = province_lr.dropna(subset=['loss_ratio'])
    plt.figure()
    sns.barplot(x='province', y='loss_ratio', data=province_lr.sort_values('loss_ratio', ascending=False), palette="viridis")
    plt.title('Loss Ratio by Province')
    plt.ylabel('Loss Ratio')
    plt.xlabel('Province')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('eda_plot_3_loss_ratio_by_province.png')
    plt.close()

    print("EDA complete. 3 plots saved: temporal trend, claim severity, and province loss ratio.")
    print("Returning cleaned dataframe for subsequent tasks.")
    return df

if __name__ == '__main__':
    # Loading the actual data file specified by the user
    initial_df = load_data(DATA_PATH)
    
    if initial_df is not None:
        cleaned_df = perform_eda(initial_df)
        
        # Save the cleaned/engineered data for the next tasks
        cleaned_df.to_csv('processed_data_for_analysis.csv', index=False)
        print("\nProcessed data saved to 'processed_data_for_analysis.csv'.")
    
# --- DVC Setup Guidance (Command Line Instructions) ---
# 1. Initialize DVC:
#    dvc init
# 2. Set Up Local Remote Storage (create the directory first):
#    mkdir .dvc_storage
#    dvc remote add -d localstorage .dvc_storage
# 3. Add Your Data (assuming you have the source file):
#    dvc add data/raw/MachineLearningRating_v3.txt
# 4. Commit .dvc files to Git:
#    git add .gitignore data/raw/MachineLearningRating_v3.txt.dvc
#    git commit -m "Add initial dataset tracked by DVC"
# 5. Push Data to Local Remote:
#    dvc push
# 6. Add the processed data:
#    dvc add processed_data_for_analysis.csv
#    git add processed_data_for_analysis.csv.dvc
#    git commit -m "Add processed data (Task 1 output)"
#    dvc push

ImportError: C extension: pandas.compat._constants not built. If you want to import pandas from the source directory, you may need to run 'python setup.py build_ext' to build the C extensions first.